In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import darts
from darts import TimeSeries, Models
import pandas as pd
import numpy as np
import time
import httpx
import json
import sqlite3
import matplotlib as plt

In [ ]:
# obtained from fuzzworks
invData = pd.read_csv('../invTypes.csv')

In [ ]:
def getMarketPrices(inplace=False):
    print('Fetching market prices')
    r = httpx.get(
        'https://esi.evetech.net/latest/markets/prices/?datasource=tranquility',)
    if (r.status_code == 200):
        df = pd.read_json(r.text)
        df['date'] = dt.datetime.now()
        con = sqlite3.connect('../data/data.db')
        df.to_sql(con=con, name="market_data", if_exists="append")
        con.close()
        if (inplace):
            return (df)
        # cur = con.cursor()
        # cur.execute('select * from market_data')
        # results = cur.fetchall()
        # print(results)
    else:
        print(f"Request did not succeed! {r.status_code}")
        rjson = json.loads(r.text)
        if (rjson['timeout'] != None or 0):
            time.sleep((rjson['timeout']/1000))
# getMarketPrices()
# adjusted_price, average_price, type_id

In [ ]:
def getItemPrices(id):
    # refresh database with new data
    getMarketPrices()
    con = sqlite3.connect('../data.db')
    # df = pd.read_sql_table(table_name='market_data',con=con,parse_dates='date')
    # cur = con.cursor()
    # cur.execute("select * from market_data")
    df = pd.read_sql('select * from market_data', con, parse_dates=['date'])
    selected_df = df.loc[df['type_id'] == id]
    if (len(selected_df) != 0):
        return (selected_df)
    else:
        print('Sorry, either this is not a valid item id or something went wrong. :P')

In [ ]:
def getIDInfo(id):
    selected_row = invData.loc[invData['typeID'] == id]
    return selected_row
# getIDInfo(32772)

In [ ]:
def getIdFromName(name):
    # NOTE: This function could be improved with a better search engine
    selected_rows = invData[invData['typeName'].str.contains(name, False)]
    selected_rows = selected_rows[['typeName', 'typeID']]
    return (selected_rows)
# getIdFromName('Booster')

In [ ]:
con = sqlite3.connect('../data.db')
databaseframe = pd.read_sql('select * from market_data', con, parse_dates=['date'])
con.close()
series = TimeSeries.from_dataframe(df=databaseframe, time_col='date')
train, val = series[:150]
model_es = Models.ExponentialSmoothing()
model_es.fit(train)
probabilistic_forecast = model_es.predict(len(val), num_samples=500)

series.plot(label="actual")
probabilistic_forecast.plot(label="probabilistic forecast")
plt.legend()
plt.show()
